# モジュールの読み込み

In [2]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import scipy as sp
import scipy.stats
import sklearn.cluster as cluster 
from sklearn.decomposition import PCA
import sklearn.tree as tree
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
import glob
import os
import csv
import io

# ファイル読み込み＆結合

In [3]:
##### レース結果の情報を読み込む関数
def read_data(path):
    ## 指定したpathに保存してあるcsvファイルをすべて読み込む
    files = glob.glob(os.path.join(path,'*.csv'))
    ## 列名を読み込むためのフラグを指定
    flag = 0
    
    df_list = []
    for file in files:
        ## 最初のファイルは普通に読み込み，カラム名を別途保存
        if flag == 0:
            tmp_df = pd.read_csv(file, encoding='cp932')
            columns = tmp_df.columns
            flag = 1
        ## 2個目以降のファイルは保存したカラム名を基に読み込みを実施
        else:
            tmp_df = pd.read_csv(file, encoding='cp932', names = columns)
            
        df_list.append(tmp_df)
    df = pd.concat(df_list, ignore_index=True,)
    ## 列名を余分に読み込んでしまっているので，除外しておく
    df = df[df['racekey'] != 'racekey']
    ## 最初は全て文字列型として読み込む
    df = df.applymap(str)
    ff = pd.DataFrame(df)
    return ff

In [4]:
%%time
## 読み込みの実施
df_race = read_data('./inputs/race_data')
df_horse = read_data('./inputs/horse_data')
df_result = read_data('./inputs/result_data')
df_just_before_info = read_data('./inputs/just_before_info')
df_practice = read_data('./inputs/practice')
df_oikiri = read_data('./inputs/oikiri')
df_other_info = read_data('./inputs/other_info')

Wall time: 2min 50s


In [5]:
##### データサイズチェック
print(df_race.shape)
print(df_horse.shape)
print(df_result.shape)
print(df_just_before_info.shape)
print(df_practice.shape)
print(df_oikiri.shape)
print(df_other_info.shape)

(464071, 36)
(464071, 125)
(464071, 84)
(464071, 34)
(464071, 34)
(464071, 29)
(464071, 27)


## ここで，最低限の前処理を行ったdfを中間テーブルとして保存しておく

In [6]:
##### 特定期間のdfを結合したdfを，中間テーブルとして保存しておく
todays_str = str(datetime.date.today())
df_race.to_csv('./intermediates/' + 'df_race_' + todays_str + '.csv')
df_horse.to_csv('./intermediates/' + 'df_horse_' + todays_str + '.csv')
df_result.to_csv('./intermediates/' + 'df_result_' + todays_str + '.csv')
df_just_before_info.to_csv('./intermediates/' + 'df_just_before_info_' + todays_str + '.csv')
df_practice.to_csv('./intermediates/' + 'df_practice_' + todays_str + '.csv')
df_oikiri.to_csv('./intermediates/' + 'df_oikiri_' + todays_str + '.csv')
df_other_info.to_csv('./intermediates/' + 'df_other_info_' + todays_str + '.csv')

## 中間テーブルからファイルを読み込めるようにしておく

In [7]:
%%time
file_dates = '2019-05-26'
df_race = pd.read_csv('./intermediates/' + 'df_race_' + file_dates + '.csv', index_col = 0)
df_horse = pd.read_csv('./intermediates/' + 'df_horse_' + file_dates + '.csv', index_col = 0)
df_result = pd.read_csv('./intermediates/' + 'df_result_' + file_dates + '.csv', index_col = 0)
df_just_before_info = pd.read_csv('./intermediates/' + 'df_just_before_info_' + file_dates + '.csv', index_col = 0)
df_practice = pd.read_csv('./intermediates/' + 'df_practice_' + file_dates + '.csv', index_col = 0)
df_oikiri = pd.read_csv('./intermediates/' + 'df_oikiri_' + file_dates + '.csv', index_col = 0)
df_other_info = pd.read_csv('./intermediates/' + 'df_other_info_' + file_dates + '.csv', index_col = 0)

<string>:2: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,7,35,36,44,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,7,64,65,66,67,75,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 31.9 s


In [8]:
##### データチェックを実施
##### memo→df_raceは不要．df_resultとdf_horseを基本的にはうまく編集していく
print(df_race.head())
print(df_horse.head())

  oldracekey           racekey 場コード   年  回  日  Ｒ       年月日  発走時間    距離  \
0    6101101  2010010506010101   中山  10  1  1  1  20100105   950  1200   
1    6101101  2010010506010101   中山  10  1  1  1  20100105   950  1200   
2    6101101  2010010506010101   中山  10  1  1  1  20100105   950  1200   
3    6101101  2010010506010101   中山  10  1  1  1  20100105   950  1200   
4    6101101  2010010506010101   中山  10  1  1  1  20100105   950  1200   

      ...        レース名９文字 データ区分 １着賞金 ２着賞金 ３着賞金 ４着賞金 ５着賞金 １着算入賞金 ２着算入賞金  \
0     ...      ３歳未勝利　　　　     4  500  200  130   75   50    400      0   
1     ...      ３歳未勝利　　　　     4  500  200  130   75   50    400      0   
2     ...      ３歳未勝利　　　　     4  500  200  130   75   50    400      0   
3     ...      ３歳未勝利　　　　     4  500  200  130   75   50    400      0   
4     ...      ３歳未勝利　　　　     4  500  200  130   75   50    400      0   

  Unnamed: 35  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  

[5 rows x 36 column

# データの前処理を実施

## 日付のカラムのデータ形式を変更

In [13]:
def change_date_columns(df, date_str):
    if '年月日' in df.columns: 
        df['年月日'] = df['年月日'].astype(str)
        df['年月日' + date_str] = pd.to_datetime(df['年月日'].str[0:4] + '-' + df['年月日'].str[4:6] + '-' + df['年月日'].str[6:8])
        df = df.drop('年月日', axis = 1)
    return df

In [14]:
df_race = change_date_columns(df_race, 'race')
df_horse = change_date_columns(df_horse, 'horse')
df_result = change_date_columns(df_result, 'result')
df_just_before_info = change_date_columns(df_just_before_info, 'jb_info')
df_practice = change_date_columns(df_practice, 'practice')
df_oikiri = change_date_columns(df_oikiri, 'oikiri')
df_other_info = change_date_columns(df_other_info, 'other_info')

## カラムの形式について，数値に変換できるものは変換する

In [15]:
## xがfloat型に変換可能かを判定する関数(flagが1なら変換可能)
def float_value_judge(x):
    float_flag = 1
    try:
        x = float(x)
    except ValueError:
        float_flag = 0
    except TypeError:
        float_flag = 0
    return float_flag

## xをfloat型に変換する関数(変換できない場合はnp.nanに強制変換)
def convert_float(x):
    try:
        x = float(x)
    except ValueError:
        x = np.nan
    return x

## 変換可能なカラム全てに対して，型をfloat型に変換する関数
def change_col_type(df):
    cols = df.columns
    df_tmp = df.copy()
    for col in cols:
        flag_list = df_tmp[col].apply(float_value_judge)
        ## flag==1の割合が99%以上である場合は，型の変換が可能であるとみなす
        if np.sum(flag_list)/len(flag_list) >= 0.99:
            df[col] = df[col].apply(convert_float)
        
    print('conversion is finished')
    return df

In [16]:
%%time
df_race = change_col_type(df_race)
df_result = change_col_type(df_result)
df_horse = change_col_type(df_horse)
df_just_before_info = change_col_type(df_just_before_info)
df_practice = change_col_type(df_practice)
df_oikiri = change_col_type(df_oikiri)
df_other_info = change_col_type(df_other_info)

conversion is finished
conversion is finished
conversion is finished
conversion is finished
conversion is finished
conversion is finished
conversion is finished
Wall time: 3min 24s


## あるdfでは文字列，別のdfでは数値となっているカラムについて，型を数値に揃える

In [17]:
def change_key_columns(df_category, df_str, target):
    ## 競馬場名とそのコードをリスト化
    str_list = df_str[target].unique()
    category_list = df_category[target].astype('float').unique()
    
    ## 競馬場名とそのコードの関係を辞書化
    target_dict = {}
    i = 0
    for s in str_list:
        target_dict[s] = category_list[i]
        i += 1
    
    ## 競馬場名が文字列で与えられているdfについて，コードへの変換を実施
    df_str[target] = df_str[target].map(target_dict)
    return df_str

In [18]:
target_columns = ['場コード', '芝ダ障害コード', '右左', '内外']

In [19]:
## 全てのdfに対して，文字列の情報を数値に変換
for target in target_columns:
    if target in df_race.columns:
        df_race = change_key_columns(df_result, df_race, target)
    if target in df_result.columns:
        df_result[target] = df_result[target].astype('float')
    if target in df_horse.columns:
        df_horse[target] = df_horse[target].astype('float')
    if target in df_just_before_info.columns:
        df_just_before_info[target] = df_just_before_info[target].astype('float')
    if target in df_practice.columns:
        df_practice[target] = df_practice[target].astype('float')
    if target in df_oikiri.columns:
        df_oikiri[target] = df_oikiri[target].astype('float')
    if target in df_other_info.columns:
        df_other_info[target] = df_other_info[target].astype('float')

In [20]:
## df_horseの「ゴール順位」は事前に予想されたゴール順を表す
## df_resultの「着順」は実際のゴール順
df_horse_test = df_horse[df_horse['kskey'] == 201904280803031000][['馬名', 'ゴール順位', 'ＩＤＭ']]
df_result_test = df_result[df_result['kskey'] == 201904280803031000][['馬名', '着順', 'ＩＤＭ']]
print(df_horse_test.head(20))
print(df_result_test.head(20))

               馬名 ゴール順位   ＩＤＭ
464711    ヴァニラアイス     8  56.2
464712     オルトグラフ     9  45.0
464713   ブルベアイリーデ     6  49.0
464714    タイミングナウ    10  53.0
464715   アヴァンティスト     5  56.1
464716  ケイアイターコイズ     7  53.0
464717    オリオンパッチ     4  56.3
464718     レッドルゼル     3  56.0
464719     モンペルデュ    13  52.0
464720   ボストンテソーロ     1  57.0
464721    グランプリワン    11  52.0
464722     ココフィーユ    15  47.0
464723  ショウナンガナドル    12  51.0
464724  ジャパンスウェプト    14  55.0
464725  ニューモニュメント     2  59.0
               馬名    着順  ＩＤＭ
464711    ヴァニラアイス   1.0   63
464712     オルトグラフ  10.0   51
464713   ブルベアイリーデ   4.0   60
464714    タイミングナウ  14.0   44
464715   アヴァンティスト   8.0   57
464716  ケイアイターコイズ   2.0   62
464717    オリオンパッチ   5.0   58
464718     レッドルゼル   7.0   56
464719     モンペルデュ   3.0   61
464720   ボストンテソーロ   6.0   57
464721    グランプリワン  13.0   45
464722     ココフィーユ  15.0   32
464723  ショウナンガナドル  12.0   48
464724  ジャパンスウェプト  11.0   48
464725  ニューモニュメント   9.0   54


# ここから特徴量の集計を開始

In [21]:
##### カラム名を確認
print(df_race.columns)
print(df_horse.columns)
print(df_result.columns)
print(df_just_before_info.columns)
print(df_practice.columns)
print(df_oikiri.columns)
print(df_other_info.columns)

Index(['oldracekey', 'racekey', '場コード', '年', '回', '日', 'Ｒ', '発走時間', '距離',
       '芝ダ障害コード', '右左', '内外', '種別', '条件', '記号1', '記号2', '記号3', '重量', 'グレード',
       'レース名', '回数', '頭数', 'コース', '開催区分', 'レース名短縮', 'レース名９文字', 'データ区分', '１着賞金',
       '２着賞金', '３着賞金', '４着賞金', '５着賞金', '１着算入賞金', '２着算入賞金', 'Unnamed: 35',
       '年月日race'],
      dtype='object')
Index(['oldkskey', 'kskey', 'racekey', '場コード', '年', '回', '日', 'Ｒ', '馬番',
       '血統登録番号',
       ...
       '馬スタート指数', '馬出遅率', '参考前走', '参考前走騎手コード', '万券指数', '万券印', '降級フラグ', '激走タイプ',
       '休養理由分類コード', 'Unnamed: 124'],
      dtype='object', length=125)
Index(['oldkskey', 'kskey', 'racekey', '場コード', '年', '回', '日', 'Ｒ', '馬番',
       '血統登録番号', '馬名', '距離', '芝ダ障害コード', '右左', '内外', '馬場状態', '種別', '条件', '記号',
       '重量', 'グレード', 'レース名', '頭数', 'レース名略称', '着順', '異常区分', 'タイム', '斤量', '騎手名',
       '調教師名', '確定単勝オッズ', '確定単勝人気順位', 'ＩＤＭ', '素点', '馬場差', 'ペース', '出遅', '位置取',
       '不利', '前不利', '中不利', '後不利', 'レース', 'コース取り', '上昇度コード', 'クラスコード', '馬体コード',
       '気配コード',

## まずは現在の日付をもとに，集計対象のレースとそれに対して遡ってデータを取得する期間を決定

In [22]:
todays_date = datetime.date.today()
train_start_date = todays_date - datetime.timedelta(days = 365 * 1)
train_end_date = todays_date - datetime.timedelta(days = 365 * 5)
print(todays_date)
print(train_start_date)
print(train_end_date)

2019-05-26
2018-05-26
2014-05-27


## 2つのデータフレームにおいて，重複するカラムを除外した上でjoinする関数を定義

In [23]:
def join_drop_duplicates(df1, df2, join_key, how_to):
    ## 2つのデータフレームに共通するカラムが，join_keyの候補となる
    duplicate_col = (df1.columns & df2.columns)
    ## join_keyとして使用しないカラムは重複してしまうので，あらかじめリストとして保持
    for element in join_key:
        duplicate_col = duplicate_col.drop(element) 
        
    ## 重複するカラムを除外
    df2 = df2.drop(duplicate_col, axis = 1)
    ## df1, df2をjoin
    df = pd.merge(df1, df2, on = join_key, how = how_to, copy = False)

## train_start_date~train_end_dateの期間のレース結果（df_result）を集計し，df_resultに追加していく

In [24]:
##### joinするデータフレームを確認しておく
df_race_base = df_race[['oldracekey', 'racekey', '場コード', '年', '回', '日', 'Ｒ', '年月日race',\
                        '距離', '芝ダ障害コード', '右左', '内外', '種別', '条件', '頭数', 'コース', '開催区分']]

df_horse_base = df_horse[['oldkskey', 'kskey', 'racekey', '場コード', '年', '回', '日', 'Ｒ', '馬番', '血統登録番号', '枠番']]#

df_result_base1 = df_result_base2 = \
                df_result[['oldkskey', 'kskey', 'racekey', '場コード', '年', '回', '日', 'Ｒ', '年月日result',\
                '馬番', '血統登録番号', '馬名', '距離', '芝ダ障害コード', '右左', '内外', '着順', '異常区分', 'タイム',\
                'コース取り', 'テン指数', '上がり指数', 'ペース指数', '1(2)着馬名', '1(2)着タイム差',\
                '前３Ｆタイム', '後３Ｆタイム', 'コーナー順位１', 'コーナー順位２', 'コーナー順位３', 'コーナー順位４',\
                '前３Ｆ先頭差', '後３Ｆ先頭差', '騎手コード', '馬体重', 'レース脚質', '４角コース取り']]
df_result_base1.head()

,oldkskey,kskey,racekey,場コード,年,回,日,Ｒ,年月日result,馬番,...,コーナー順位１,コーナー順位２,コーナー順位３,コーナー順位４,前３Ｆ先頭差,後３Ｆ先頭差,騎手コード,馬体重,レース脚質,４角コース取り
0,610110101,2.010011e+17,2.010011e+15,6.0,10.0,1.0,1,1.0,2010-01-05,1.0,...,0.0,11.0,10.0,8.0,-10,-10,10539.0,436.0,差,内
1,610110102,2.010011e+17,2.010011e+15,6.0,10.0,1.0,1,1.0,2010-01-05,2.0,...,0.0,13.0,13.0,11.0,-13,-13,10442.0,470.0,追,内
2,610110103,2.010011e+17,2.010011e+15,6.0,10.0,1.0,1,1.0,2010-01-05,3.0,...,0.0,8.0,12.0,13.0,-11,-11,10528.0,424.0,追,最
3,610110104,2.010011e+17,2.010011e+15,6.0,10.0,1.0,1,1.0,2010-01-05,4.0,...,0.0,12.0,10.0,9.0,-10,-10,10542.0,428.0,差,中
4,610110105,2.010011e+17,2.010011e+15,6.0,10.0,1.0,1,1.0,2010-01-05,5.0,...,0.0,8.0,6.0,5.0,-6,-6,10462.0,460.0,差,最


In [28]:
#####race_dataについては，期間を一定に絞る & 障害レースを除外
df_result_base1 = df_result_base1[(df_result_base1['年月日result'] > pd.to_datetime(train_end_date)) & \
                                 (df_result_base1['年月日result'] < pd.to_datetime(train_start_date))]

## データサイズのチェック
print(df_result_base1.shape) ## 期間を絞った後
print(df_result_base2.shape) ## 全期間
print(df_result_base1[df_result_base1['芝ダ障害コード'] == 3])

(198838, 37)
(464071, 37)
          oldkskey         kskey       racekey  場コード     年    回  日    Ｒ  \
221003  08143b0801  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221004  08143b0802  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221005  08143b0803  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221006  08143b0804  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221007  08143b0805  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221008  08143b0806  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221009  08143b0807  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221010  08143b0808  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221011  08143b0809  2.014053e+17  2.014053e+15   8.0  14.0  3.0  b  8.0   
221312  08143c0401  2.014060e+17  2.014060e+15   8.0  14.0  3.0  c  4.0   
221313  08143c0402  2.014060e+17  2.014060e+15   8.0  14.0  3.0  c  4.0   
221314  08143c0403  2.014060e+17  2.014060e+15   8.0  14.0  3.0  c  4.0   

In [ ]:
df_result_aggregate_tmp = join_drop_duplicates(df_result_base1, df_result_base2,\
                                               ['場コード', '距離', '芝ダ障害コード', '馬番'], 'left')

## 競走馬の過去出走データの集計テスト

In [ ]:
df_horse_test = df_horse[(df_horse['racekey'] == '2019042808030311')]
df_horse_test = df_horse_test[['racekey', '馬名', '前走１競走成績キー', '前走２競走成績キー']]

In [ ]:
#df_race_data = df[['race_key', '場コード', '馬番', '血統登録番号', '年月日', '馬名', '距離', '芝ダ障害コード', '右左', '内外',\
#                  '条件', '重量', 'レース名', '頭数', '斤量']]
df_horse_test.head()

In [ ]:
df_result['join_key'] = df_result['血統登録番号'] + df_result['年月日']
df_result_test = df_result[['join_key', 'レース名', '着順']]
df_horse_race_test = pd.merge(df_horse_test, df_result_test,\
                              left_on = '前走１競走成績キー', right_on = 'join_key', how = 'left')
df_horse_race_test.head()